In [2]:
import numpy as np
import pandas as pd
from astropy.time import Time
import h5py
from pycbc.conversions import mchirp_from_mass1_mass2,primary_mass,secondary_mass,chi_eff
import pylab,os,shutil
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',None)

# Select GW triggers

In [3]:
#4-OGC triggers are released in https://github.com/gwastro/4-ogc
fourogc = h5py.File('/work/ahnitz/WWW/4ogc/4-ogc.hdf')

In [4]:
gwfile = fourogc

# convert the GW events name to UTF-8 format
gwobs = np.array([obs.decode("utf-8") for obs in gwfile['trig'][:]])

mass1 = np.array(gwfile['mass1'])
mass2 = np.array(gwfile['mass2'])
spin1z = np.array(gwfile['spin1z'])
spin2z = np.array(gwfile['spin2z'])
spin = chi_eff(gwfile['mass1'][:], gwfile['mass2'][:],gwfile['spin1z'][:], gwfile['spin2z'][:])
name_gw = np.array(gwfile['name'])
ifar = np.array(gwfile['ifar'])
time_gw = np.array(gwfile['time'])

In [5]:
lmass = (mass2<=2) & (mass1<=2)
lspin = (spin<0.2) & (spin>-0.2)

#at least two detectors
# This doesn't affect the results because single detector trigger doesn't have a IFAR
ltrig = (gwobs == 'HL') | (gwobs == 'HLV') | (gwobs == 'LV') | (gwobs == 'HV')

# time during O3a start and 2019 07, 02 00:00:00
ltime = (gwfile['time'][:] > Time('2019-04-01T00:00:00').gps) & (gwfile['time'][:] < Time('2019-07-02T00:00:00').gps)
l = lmass & lspin & ltrig & ltime
print('There are ', l.sum(), ' triggers')

# the index of a qualified GW event
gwidx = np.where(l)[0]

There are  294489  triggers


In [6]:
gwobs = gwobs[gwidx]
mass1 = mass1[gwidx]
mass2 = mass2[gwidx]
spin1z = spin1z[gwidx]
spin2z = spin2z[gwidx]
name_gw = name_gw[gwidx]
ifar = ifar[gwidx]
time_gw = time_gw[gwidx]

# select FRB events

In [7]:
#FRB file: CHIME FRB catalog 1
#This can be downloaded from https://www.chime-frb.ca/catalog
frb = pd.read_csv('/work/yifan.wang/em/gitrepo/chimefrbcat1.csv')
# convert the mjd_inf time to a datetime64 format
frb_t =  Time(frb['mjd_inf'][:],format='mjd').datetime64

tO3_start = Time('2019-04-01T00:00:00')
tO3_end = Time('2019-07-02T00:00:00')

frb = frb[(frb_t <tO3_end) & (frb_t > tO3_start) & (frb['repeater_name'] == '-9999')] #not a repeated FRB

In [8]:
#add a new column called 'gpstime'
frb['gpstime'] = Time(frb['mjd_inf'][:],format='mjd').gps

#make four arrays which will be useful
t_frb = np.array(frb['gpstime'])
name_frb = np.array(frb['tns_name'])
ra_frb = np.array(frb['ra'])/ 180 * np.pi
dec_frb = np.array(frb['dec']) / 180 * np.pi

# There are 151 FRB events

In [9]:
name_frb, uid = np.unique(name_frb, return_index=True)
ra_frb = ra_frb[uid]
dec_frb = dec_frb[uid]
t_frb = t_frb[uid]

In [10]:
sortidx = np.argsort(t_frb)

In [11]:
t_frb = t_frb[sortidx]
ra_frb = ra_frb[sortidx]
dec_frb = dec_frb[sortidx]
name_frb = name_frb[sortidx]

# Make coincident triggers: Time window [-100,100]

In [12]:
#[t_frb ---100s--- t_gw ----------- 100s ------------- t_frb]
left = np.searchsorted(t_frb,  time_gw- 100)
right = np.searchsorted(t_frb, time_gw + 100)

# The index of 'left' is GW index, the value of 'left' is frb index
gw_co_idx = np.where((right - left) > 0)[0]
frb_co_idx = left[gw_co_idx]
print('There are ', len(gw_co_idx), 'coincidence.')

There are  1050 coincidence.


In [15]:
for ii in gw_co_idx:
    print('GW trigger',name_gw[ii],'associates with', name_frb[left[ii]])

GW trigger b'190403_023537' associates with FRB20190403A
GW trigger b'190403_023559' associates with FRB20190403A
GW trigger b'190403_023610' associates with FRB20190403A
GW trigger b'190403_023639' associates with FRB20190403A
GW trigger b'190403_023702' associates with FRB20190403A
GW trigger b'190403_023746' associates with FRB20190403A
GW trigger b'190403_023816' associates with FRB20190403A
GW trigger b'190403_023841' associates with FRB20190403A
GW trigger b'190403_041823' associates with FRB20190403B
GW trigger b'190403_041850' associates with FRB20190403B
GW trigger b'190403_041914' associates with FRB20190403B
GW trigger b'190403_041927' associates with FRB20190403B
GW trigger b'190403_041947' associates with FRB20190403B
GW trigger b'190403_042014' associates with FRB20190403B
GW trigger b'190403_042040' associates with FRB20190403B
GW trigger b'190403_042056' associates with FRB20190403B
GW trigger b'190403_042116' associates with FRB20190403B
GW trigger b'190403_114651' ass

GW trigger b'190606_221803' associates with FRB20190606B
GW trigger b'190606_221818' associates with FRB20190606B
GW trigger b'190606_221842' associates with FRB20190606B
GW trigger b'190606_221905' associates with FRB20190606B
GW trigger b'190606_221916' associates with FRB20190606B
GW trigger b'190606_221941' associates with FRB20190606B
GW trigger b'190606_221958' associates with FRB20190606B
GW trigger b'190606_222010' associates with FRB20190606B
GW trigger b'190606_222031' associates with FRB20190606B
GW trigger b'190606_222047' associates with FRB20190606B
GW trigger b'190606_222110' associates with FRB20190606B
GW trigger b'190607_163745' associates with FRB20190607A
GW trigger b'190607_163801' associates with FRB20190607A
GW trigger b'190607_163819' associates with FRB20190607A
GW trigger b'190607_163844' associates with FRB20190607A
GW trigger b'190607_163906' associates with FRB20190607A
GW trigger b'190607_163922' associates with FRB20190607A
GW trigger b'190607_163938' ass

# Make background: Time window [-100,100]

In [16]:
#time sliding one hundred times
n = 100
step = (tO3_end.gps - tO3_start.gps) / (n+1)


for nvalue in range(n):
    print('chunk:',nvalue)
    
    offset = (nvalue + 1) * step
    t_frb_timeslide = (t_frb + offset) % tO3_end.gps + (t_frb+offset)//tO3_end.gps * tO3_start.gps
    
    # sort the t_frb
    sortfrb = np.argsort(t_frb_timeslide)
    t_frb_timeslide = t_frb_timeslide[sortfrb]
    ra_timeslide = ra_frb[sortfrb]
    dec_timeslide = dec_frb[sortfrb]
    #[t_frb ---100s--- t_gw ----------- 100s ------------- t_frb]
    left = np.searchsorted(t_frb_timeslide, time_gw - 100)
    right = np.searchsorted(t_frb_timeslide, time_gw + 100)

    gw_co_idx = np.where((right - left) > 0)[0]
    frb_co_idx = left[gw_co_idx]
    print('There are ', len(gw_co_idx), 'coincidence.')

chunk: 0
There are  1086 coincidence.
chunk: 1
There are  1125 coincidence.
chunk: 2
There are  1085 coincidence.
chunk: 3
There are  1165 coincidence.
chunk: 4
There are  1223 coincidence.
chunk: 5
There are  1032 coincidence.
chunk: 6
There are  1086 coincidence.
chunk: 7
There are  1183 coincidence.
chunk: 8
There are  1102 coincidence.
chunk: 9
There are  1097 coincidence.
chunk: 10
There are  1157 coincidence.
chunk: 11
There are  1059 coincidence.
chunk: 12
There are  1174 coincidence.
chunk: 13
There are  1007 coincidence.
chunk: 14
There are  1107 coincidence.
chunk: 15
There are  971 coincidence.
chunk: 16
There are  1025 coincidence.
chunk: 17
There are  1166 coincidence.
chunk: 18
There are  1146 coincidence.
chunk: 19
There are  1131 coincidence.
chunk: 20
There are  1033 coincidence.
chunk: 21
There are  1109 coincidence.
chunk: 22
There are  1141 coincidence.
chunk: 23
There are  1183 coincidence.
chunk: 24
There are  1128 coincidence.
chunk: 25
There are  1123 coincidenc